# Аугментация mix и сглаженная

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

## Настройка загрузчика данных DataLoaders

Мы будем использовать встроенный набор данных `torchvision.datasets.ImageFolder`, чтобы быстро создать несколько загрузчиков данных загруженных изображений кошек и рыб. 

`check_image` - это небольшая функция, которая передается параметру `is_valid_file` в ImageFolder и выполняет проверку на вменяемость, чтобы убедиться, что PIL действительно может открыть файл. Мы будем использовать ее вместо очистки загруженного набора данных.

In [2]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

Установка трансформации для каждого изображения:

* Ужатие до 64x64
* Преобразование в тензор
* Нормализация с использованием параметров ImageNet mean & std


In [3]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])



## Распаковка датасета

In [18]:
!unzip images

Archive:  images.zip
   creating: test/
   creating: test/fish/
  inflating: test/fish/2242992316_2d999508d6.jpg  
  inflating: test/fish/2163764976_f78d9d0f86.jpg  
  inflating: test/fish/359504247_abafe8992c.jpg  
  inflating: test/fish/578651623_d4cdbc4846.jpg  
  inflating: test/fish/2166804428_663d7a094b.jpg  
  inflating: test/fish/120994748_bf47e3da6d.jpg  
  inflating: test/fish/1332343289_a3a43fafe6.jpg  
  inflating: test/fish/1394413521_27536c0a8f.jpg  
  inflating: test/fish/50081802_bf4d207cd5.jpg  
 extracting: test/fish/silversalmon.jpg  
  inflating: test/fish/3046908845_10016c4dcd.jpg  
  inflating: test/fish/999386881_b6bbc2c39a.jpg  
  inflating: test/fish/7654_1163794241.jpg  
 extracting: test/fish/sctrnbw06_07.jpg  
 extracting: test/fish/dave%20with%20two%20silvers.jpg  
  inflating: test/fish/2561312476_c39a0095c4.jpg  
 extracting: test/fish/Coho%20Salmon%20%20IMG_0970.JPG  
 extracting: test/fish/2919672812_a2deec6048.jpg  
  inflating: test/fish/194363226_298

In [19]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [20]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [21]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 

In [22]:
batch_size=64

In [23]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

## Модель нейронной сети 



In [24]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [25]:
simplenet = CNNNet()

## Создаем оптимизатор

Здесь мы просто используем Adam в качестве оптимизатора с коэффициентом обучения 0,001.

In [26]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Копирование модели на ГПУ

Копирование модели на ГПУ в случае возможности

In [27]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

CNNNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4)

## Обучение

Обучаем модель, при необходимости копируя батчи на GPU, рассчитывая потери, оптимизируя сеть и выполняя валидацию для каждой эпохи.

In [28]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [29]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 1601.95, Validation Loss: 0.72, accuracy = 0.18
Epoch: 2, Training Loss: 0.71, Validation Loss: 0.73, accuracy = 0.18
Epoch: 3, Training Loss: 0.67, Validation Loss: 1.00, accuracy = 0.18
Epoch: 4, Training Loss: 0.78, Validation Loss: 0.74, accuracy = 0.18
Epoch: 5, Training Loss: 0.69, Validation Loss: 0.79, accuracy = 0.18


## Предсказание

Метки располагаются в буквенно-цифровом порядке, поэтому `cat` будет 0, `fish` - 1. Нам понадобится преобразовать изображение, а также убедиться, что полученный тензор скопирован на соответствующее устройство, прежде чем применять к нему нашу модель.

In [33]:
labels = ['cat','fish']

img = Image.open("./val/cat/1230938727_7eaf8a8718.jpg") 
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction]) 

fish


## Сохранение модели

Мы можем сохранить либо всю модель с помощью save, либо только параметры с помощью state_dict. Последний вариант обычно предпочтительнее, так как позволяет повторно использовать параметры даже при изменении структуры модели (или применять параметры от одной модели к другой).

In [ ]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")    


In [ ]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = CNNNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)   

## mix

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device, mix_loader):
  for epoch in range(epochs):
    model.train()
    for batch in zip(train_loader,mix_loader):
      ((inputs, targets),(inputs_mix, targets_mix)) = batch
      optimizer.zero_grad()
      inputs = inputs.to(device)
      targets = targets.to(device)
      inputs_mix = inputs_mix.to(device)
      target_mix = targets_mix.to(device)
      distribution = torch.distributions.beta.Beta(0.5,0.5)
      beta = distribution.expand(torch.zeros(
      batch_size).shape).sample().to(device)
      # Нам нужно преобразовать форму бета, 
      # чтобы она была в тех же размерностях, что и наш входной тензор
      # [batch_size, channels, height, width]
      mixup = beta[:, None, None, None]
      inputs_mixed = (mixup * inputs) + (1-mixup * inputs_mix)
      # Targets are mixed using beta as they have the same shape
      targets_mixed = (beta * targets) + (1-beta * inputs_mix)
      output_mixed = model(inputs_mixed)
      # Умножьте потери на бета и 1-бета, 
      # сложите и получите среднее значение из двух смешанных потерь
      loss = (loss_fn(output, targets) * beta + loss_fn(output, targets_mixed) * (1-beta)).mean()
      # Метод обучения, в обычном режиме, здесь и далее
      loss.backward()
      optimizer.step()

## Сглаживание маркировок

In [ ]:
сlass LabelSmoothingCrossEntropyLoss(nn.Module):
  def __init__(self, epsilon=0.1):
    super(LabelSmoothingCrossEntropyLoss, self).__init__()
    self.epsilon = epsilon
    
  def forward(self, output, target):
    num_classes = output.size()[-1]
    log_preds = F.log_softmax(output, dim=-1)
    loss = (-log_preds.sum(dim=-1)).mean()
    nll = F.nll_loss(log_preds, target)
    final_loss = self.epsilon * loss / num_classes + (1-self.epsilon) * nll
    return final_loss